In [1]:
import numpy as np
import pandas as pd
import pickle

In [5]:
# 1 Data 
data = pickle.load(open('data_face_features.pickle',mode='rb'))

In [6]:
X = np.array(data['data'])
y = np.array(data['label'])

In [7]:
X.shape,y.shape

((1655, 1, 128), (1655,))

In [8]:
X = X.reshape(-1,128)
X.shape

(1655, 128)

In [9]:
# split data into train and test
from sklearn.model_selection import train_test_split

In [10]:
x_train,x_test,y_train,y_test = train_test_split(X,y,train_size=0.8,random_state = 0)

In [11]:
x_train.shape,x_test.shape,y_train.shape,y_test.shape

((1324, 128), (331, 128), (1324,), (331,))

In [12]:
# 2 Train Machine learning model

In [13]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import classification_report,accuracy_score,f1_score

In [14]:
# logistic Regression
model_logistic = LogisticRegression()
model_logistic.fit(x_train,y_train) # training logistic regression

LogisticRegression()

In [19]:
def get_report(model,x_train,y_train,x_test,y_test):
    y_pred_train = model.predict(x_train)
    y_pred_test = model.predict(x_test)
    
    # accuracy score
    acc_train = accuracy_score(y_train,y_pred_train)
    acc_test = accuracy_score(y_test,y_pred_test)
    
    # f1 score
    f1_score_train = f1_score(y_train,y_pred_train,average='macro')
    f1_score_test = f1_score(y_test,y_pred_test,average='macro')
    
    print('Accuracy Train = %0.2f'%acc_train)
    print('Accuracy Test = %0.2f'%acc_test)
    print('F1 Score Train = %0.2f'%f1_score_train)
    print('F1 Score Test = %0.2f'%f1_score_test)

In [21]:
get_report(model_logistic,x_train,y_train,x_test,y_test)

Accuracy Train = 0.75
Accuracy Test = 0.62
F1 Score Train = 0.75
F1 Score Test = 0.60


In [22]:
# Support vector machine
model_svc = SVC(probability=True)
model_svc.fit(x_train,y_train)

SVC(probability=True)

In [23]:
get_report(model_svc,x_train,y_train,x_test,y_test)

Accuracy Train = 0.84
Accuracy Test = 0.66
F1 Score Train = 0.84
F1 Score Test = 0.65


In [28]:
# Random Forest
model_rf = RandomForestClassifier(n_estimators=10,)
model_rf.fit(x_train,y_train)

RandomForestClassifier(n_estimators=10)

In [29]:
get_report(model_rf,x_train,y_train,x_test,y_test)

Accuracy Train = 0.99
Accuracy Test = 0.57
F1 Score Train = 0.99
F1 Score Test = 0.56


In [36]:
# Voting Classifier
model_voting = VotingClassifier(estimators=[
    ('logistic',LogisticRegression()),
    ('svm',SVC(probability=True)),
     ('rf',RandomForestClassifier())
], voting ='soft',weights=[2,3,1])

In [37]:
model_voting.fit(x_train,y_train)

VotingClassifier(estimators=[('logistic', LogisticRegression()),
                             ('svm', SVC(probability=True)),
                             ('rf', RandomForestClassifier())],
                 voting='soft', weights=[2, 3, 1])

In [39]:
get_report(model_voting,x_train,y_train,x_test,y_test)

Accuracy Train = 0.89
Accuracy Test = 0.65
F1 Score Train = 0.89
F1 Score Test = 0.64


In [40]:
# parameter tuning

In [42]:
from sklearn.model_selection import GridSearchCV


In [43]:
model_grid = GridSearchCV(model_voting,
                          param_grid={
                              'svm__C':[3,5,7,10],
                              'svm__gamma':[0.1,0.3,0.5],
                              'rf__n_estimators':[5,10,20],
                              'rf__max_depth':[3,5,7],
                              'voting':['soft','hard']
                          },scoring='accuracy',cv=3,n_jobs=1,verbose=2)

In [44]:
model_grid.fit(x_train,y_train)

Fitting 3 folds for each of 216 candidates, totalling 648 fits
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=soft; total time=   0.9s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=soft; total time=   0.9s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=soft; total time=   0.7s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=hard; total time=   0.9s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=hard; total time=   1.3s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=hard; total time=   2.4s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.3, voting=soft; total time=   0.8s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.3, voting=soft; total time=   0.8s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.3, voting=soft; total time=   0.7s
[

GridSearchCV(cv=3,
             estimator=VotingClassifier(estimators=[('logistic',
                                                     LogisticRegression()),
                                                    ('svm',
                                                     SVC(probability=True)),
                                                    ('rf',
                                                     RandomForestClassifier())],
                                        voting='soft', weights=[2, 3, 1]),
             n_jobs=1,
             param_grid={'rf__max_depth': [3, 5, 7],
                         'rf__n_estimators': [5, 10, 20],
                         'svm__C': [3, 5, 7, 10], 'svm__gamma': [0.1, 0.3, 0.5],
                         'voting': ['soft', 'hard']},
             scoring='accuracy', verbose=2)

In [45]:
model_best_estimator = model_grid.best_estimator_

In [46]:
model_grid.best_score_

0.7182565333825838

In [49]:
# save model
pickle.dump(model_best_estimator,open('./models/machinelearning_face_person_identity.pkl',mode='wb'))